In [1]:
import aux_functions
from app_functions import attempt_predict
from transformers import TimeTransformer, BoolTransformer

import json
import joblib
import pickle
import requests
from time import sleep
import random

import pandas as pd
pd.set_option('display.max_columns', 100)
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score
# from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, RobustScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.utils import resample
from sklearn.metrics import precision_score, recall_score, f1_score,  accuracy_score, roc_auc_score, make_scorer, confusion_matrix, roc_curve

# needed to use matplotlib inside jupyter notebook
%matplotlib inline 

# Get the data

In [2]:
df_ = pd.read_csv("data/train.csv")

drop_cols = ['Self-defined ethnicity', 'Outcome', 'Outcome linked to object of search', 'Removal of more than just outer clothing']


df_clean = aux_functions.clean_data(df_, drop_cols)

df_train, df_test = train_test_split(df_clean, test_size=0.3, random_state=42)

df_test.head()

,observation_id,Type,Date,Part of a policing operation,Latitude,Longitude,Gender,Age range,Officer-defined ethnicity,Legislation,Object of search,station,target
438050,b7287b3c-fb25-42a5-afa3-e8320817eb6f,Person search,2021-07-30T16:00:00+00:00,False,53.797198,-1.789092,Male,25-34,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,west-yorkshire,0
8521,4f9c9bbb-8806-4a9f-a83d-a7d1c9957527,Person search,2020-12-20T01:26:43+00:00,<NA>,51.508963,-0.073894,Male,over 34,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,city-of-london,0
601361,1e57860f-9cb6-4afe-aabf-00b11c5012d2,Person search,2021-05-12T13:05:27+00:00,<NA>,53.397291,-3.034496,Male,over 34,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,merseyside,0
282181,8d166c18-7351-41e0-8be6-3f9fb574d8d9,Person search,2020-06-15T11:20:00+00:00,<NA>,53.041893,-2.979160,Male,over 34,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,north-wales,0
797640,3a2bdbf8-83f2-448b-bcf7-a2502632d5a8,Person search,2020-08-13T23:00:00+00:00,False,51.022386,-0.353517,Male,10-17,White,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,sussex,0


# Recover Model

In [3]:
with open(os.path.join("columns.json"), 'r') as fh:
    columns = json.load(fh)

with open(os.path.join("dtypes.pickle"), 'rb') as fh:
    dtypes = pickle.load(fh)

pipeline_recovered = joblib.load(os.path.join("pipeline.pickle"))

# Define APP Protection Function

## Objective

* Validate that observation includes a valid id
* Validate that observation has no unknown fields
* Validate that fields have the correct data types
* Validate that categorical columns have correct values


In [4]:
df_test.columns

Index(['observation_id', 'Type', 'Date', 'Part of a policing operation',
       'Latitude', 'Longitude', 'Gender', 'Age range',
       'Officer-defined ethnicity', 'Legislation', 'Object of search',
       'station', 'target'],
      dtype='object')

# Generate Observation

In [5]:
y_test = df_test["target"].copy()
X_test = df_test.drop(columns=["target"]).copy()

In [6]:
obs_dict = X_test.iloc[0, :].to_dict()

obs_dict

{'observation_id': 'b7287b3c-fb25-42a5-afa3-e8320817eb6f',
 'Type': 'Person search',
 'Date': '2021-07-30T16:00:00+00:00',
 'Part of a policing operation': False,
 'Latitude': 53.797198,
 'Longitude': -1.789092,
 'Gender': 'Male',
 'Age range': '25-34',
 'Officer-defined ethnicity': 'Asian',
 'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
 'Object of search': 'Controlled drugs',
 'station': 'west-yorkshire'}

In [7]:
# example with extra columns

obs_dict = {'observation_id': "b7287b3c-fb25-42a5-afa3-e8320817eb6f",
    'Type': 'Person search',
    'Date': '2021-07-30T16:00:00+00:00',
    'Part of a policing operation': False,
    'Latitude': 53.797198,
    'Longitude': -1.789092,
    'Gender': 'Male',
    'extra column 1': 1,
    'extra column 2': 3,
    'Age range': '25-34',
    'Officer-defined ethnicity': 'Asian',
    'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
    'Object of search': 'Controlled drugs',
    'station': 'west-yorkshire'}

response, check = aux_functions.attempt_predict(obs_dict)

if check:
    print("passed")
    display(response)
else: 
    display(response)

{'error': "Unrecognized columns provided: {'extra column 2', 'extra column 1'}"}

In [8]:
# example with missing columns

obs_dict = {'observation_id': "b7287b3c-fb25-42a5-afa3-e8320817eb6f",
    'Type': 'Person search',
    'Date': '2021-07-30T16:00:00+00:00',
    'Part of a policing operation': False,
    'Latitude': 53.797198,
    'Age range': '25-34',
    'Officer-defined ethnicity': 'Asian',
    'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
    'Object of search': 'Controlled drugs',
    'station': 'west-yorkshire'}

response, check = aux_functions.attempt_predict(obs_dict)

if check:
    print("passed")
    display(response)
else: 
    display(response)

{'error': "Missing columns: {'Gender', 'Longitude'}"}

In [9]:
# Example with weird Type category

obs_dict = {'observation_id': "b7287b3c-fb25-42a5-afa3-e8320817eb6f",
    'Type': 'Alien Invaders',
    'Date': '2021-07-30T16:00:00+00:00',
    'Part of a policing operation': False,
    'Latitude': 53.797198,
    'Longitude': -1.789092,
    'Gender': 'Male',
    'Age range': '25-34',
    'Officer-defined ethnicity': 'Asian',
    'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
    'Object of search': 'Controlled drugs',
    'station': 'west-yorkshire'}

response, check = aux_functions.attempt_predict(obs_dict)

if check:
    print("passed")
    display(response)
else: 
    display(response)


{'error': "Invalid value provided for Type: Alien Invaders. Allowed values are: 'Person search','Person and Vehicle search','Vehicle search'"}

In [10]:
# Example with weird Gender category

obs_dict = {'observation_id': "b7287b3c-fb25-42a5-afa3-e8320817eb6f",
    'Type': 'Person search',
    'Date': '2021-07-30T16:00:00+00:00',
    'Part of a policing operation': False,
    'Latitude': 53.797198,
    'Longitude': -1.789092,
    'Gender': 'Whatever',
    'Age range': '25-34',
    'Officer-defined ethnicity': 'Asian',
    'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
    'Object of search': 'Controlled drugs',
    'station': 'west-yorkshire'}

response, check = aux_functions.attempt_predict(obs_dict)

if check:
    print("passed")
    display(response)
else: 
    display(response)


{'error': "Invalid value provided for Gender: Whatever. Allowed values are: 'Female','Male','Other'"}

In [11]:
# Example with weird Age range category

obs_dict = {'observation_id': "b7287b3c-fb25-42a5-afa3-e8320817eb6f",
    'Type': 'Person search',
    'Date': '2021-07-30T16:00:00+00:00',
    'Part of a policing operation': False,
    'Latitude': 53.797198,
    'Longitude': -1.789092,
    'Gender': 'Male',
    'Age range': '10000000000000>',
    'Officer-defined ethnicity': 'Asian',
    'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
    'Object of search': 'Controlled drugs',
    'station': 'west-yorkshire'}

response, check = aux_functions.attempt_predict(obs_dict)

if check:
    print("passed")
    display(response)
else: 
    display(response)

{'error': "Invalid value provided for Age range: 10000000000000>. Allowed values are: 'under 10','10-17','18-24','25-34','over 34'"}

In [12]:
# Example with weird Officer-defined ethnicity category

obs_dict = {'observation_id': "b7287b3c-fb25-42a5-afa3-e8320817eb6f",
    'Type': 'Person search',
    'Date': '2021-07-30T16:00:00+00:00',
    'Part of a policing operation': False,
    'Latitude': 53.797198,
    'Longitude': -1.789092,
    'Gender': 'Male',
    'Age range': '25-34',
    'Officer-defined ethnicity': 'Extra-terrestrial',
    'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
    'Object of search': 'Controlled drugs',
    'station': 'west-yorkshire'}

response, check = aux_functions.attempt_predict(obs_dict)

if check:
    print("passed")
    display(response)
else: 
    display(response)


{'error': "Invalid value provided for Officer-defined ethnicity: Extra-terrestrial. Allowed values are: 'Asian','Black','Mixed','White','Other'"}

In [13]:
# Example with weird date ethnicity category

obs_dict = {'observation_id': "b7287b3c-fb25-42a5-afa3-e8320817eb6f",
    'Type': 'Person search',
    'Date': 'yes',
    'Part of a policing operation': False,
    'Latitude': 53.797198,
    'Longitude': -1.789092,
    'Gender': 'Male',
    'Age range': '25-34',
    'Officer-defined ethnicity': 'Black',
    'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
    'Object of search': 'Controlled drugs',
    'station': 'west-yorkshire'}

response, check = aux_functions.attempt_predict(obs_dict)

if check:
    print("passed")
    display(response)
else: 
    display(response)

{'Date': 'yes', 'error': 'Date format is incorrect'}

In [14]:
# Example with date before 2020 ethnicity category

obs_dict = {'observation_id': "b7287b3c-fb25-42a5-afa3-e8320817eb6f",
    'Type': 'Person search',
    'Date': '2019-07-30T16:00:00+00:00',
    'Part of a policing operation': False,
    'Latitude': 53.797198,
    'Longitude': -1.789092,
    'Gender': 'Male',
    'Age range': '25-34',
    'Officer-defined ethnicity': 'Black',
    'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
    'Object of search': 'Controlled drugs',
    'station': 'west-yorkshire'}

response, check = aux_functions.attempt_predict(obs_dict)

if check:
    print("passed")
    display(response)
else: 
    display(response)

{'Date': '2019-07-30T16:00:00+00:00', 'error': 'Provided date is before 2020'}

# Run through test data set

In [16]:
# reshuffle available test set
r = list(range(len(X_test)))
random.shuffle(r)

for i in r:
    
    obs_dict = X_test.iloc[i, :].to_dict()

    response, check = aux_functions.attempt_predict(obs_dict)

    if check:
        print("passed")
        display(response)
    else: 
        display(response)
    
    sleep(0.5)

passed


{'observation_id': 'c871fe09-ea1a-4f7c-96e0-9b0c951ef9f7',
 'Type': 'Person search',
 'Date': '2021-03-31T17:00:00+00:00',
 'Part of a policing operation': False,
 'Latitude': 52.277093,
 'Longitude': -1.532447,
 'Gender': 'Male',
 'Age range': 'over 34',
 'Officer-defined ethnicity': 'Mixed',
 'Legislation': 'Police and Criminal Evidence Act 1984 (section 1)',
 'Object of search': 'Anything to threaten or harm anyone',
 'station': 'warwickshire'}

{'Part of a policing operation': <NA>,
 'error': 'Provided "Part of a policing operation" field is not of the correct data type'}

passed


{'observation_id': '98c66d19-f859-4f00-974e-aed0080e31dd',
 'Type': 'Person and Vehicle search',
 'Date': '2021-02-19T19:40:00+00:00',
 'Part of a policing operation': False,
 'Latitude': 53.714726,
 'Longitude': -1.882627,
 'Gender': 'Male',
 'Age range': '25-34',
 'Officer-defined ethnicity': 'White',
 'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
 'Object of search': 'Controlled drugs',
 'station': 'west-yorkshire'}

...

{'observation_id': 'd006d4f3-8b0a-4317-86c9-d2683fec84a6',
 'Type': 'Person and Vehicle search',
 'Date': '2021-07-07T22:30:00+00:00',
 'Part of a policing operation': False,
 'Latitude': 51.897327,
 'Longitude': -2.070903,
 'Gender': 'Male',
 'Age range': '10-17',
 'Officer-defined ethnicity': 'White',
 'Legislation': 'Firearms Act 1968 (section 47)',
 'Object of search': 'Firearms',
 'station': 'gloucestershire'}

{'Part of a policing operation': <NA>,
 'error': 'Provided "Part of a policing operation" field is not of the correct data type'}

{'Part of a policing operation': <NA>,
 'error': 'Provided "Part of a policing operation" field is not of the correct data type'}

{'Part of a policing operation': <NA>,
 'error': 'Provided "Part of a policing operation" field is not of the correct data type'}

KeyboardInterrupt: 

In [23]:
r = list(range(len(X_test)))
test_list = random.shuffle(r)

url = f"http://{APP_NAME}/should_search"
payload = {  
        'observation_id': '2e4d0094-c30b-471b-a211-72a9790feca2',
        'Type': 'Person search',
        'Date': '2020-12-01 01:10:00+0000',
        'Part of a policing operation': False,
        'Latitude': 50.798824,
        'Longitude': -1.089471,
        'Gender': 'Male',
        'Age range': '25-34',
        'Officer-defined ethnicity': 'White',
        'Legislation': 'Police and Criminal Evidence Act 1984 (section 1)',
        'Object of search': 'Article for use in theft',
        'station': 'hampshire'
    }

r = requests.post(url, json=payload)
display(r)
print(r.content)

<Response [200]>

b'{\n  "error": "Observation ID: \\"2e4d0094-c30b-471b-a211-72a9790feca2\\" already exists", \n  "outcome": "[False]"\n}\n'


In [22]:
# TEST LOCALLY

APP_NAME = 'localhost:5000' 


In [34]:
r = list(range(len(X_test)))
random.shuffle(r)

url = f"http://{APP_NAME}/should_search"

for i in r:
    
    obs_dict = X_test.iloc[i, :].to_dict()

    payload = obs_dict

    display(payload)

    r = requests.post(url, json=payload)
    display(r)
    print(r.content)
    
    sleep(0.5)

{'observation_id': '45ae789b-68ce-4197-a4a3-d09427f47500',
 'Type': 'Person search',
 'Date': '2020-07-07T17:30:00+00:00',
 'Part of a policing operation': <NA>,
 'Latitude': 53.20278,
 'Longitude': -2.870975,
 'Gender': 'Male',
 'Age range': '25-34',
 'Officer-defined ethnicity': 'White',
 'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
 'Object of search': 'Controlled drugs',
 'station': 'cheshire'}

TypeError: Object of type NAType is not JSON serializable